In [15]:
from pyspark import SparkConf, SparkContext
from os import listdir
from os.path import isfile, join

In [22]:
STOPSET = {'they', 'she', 'he', 'it', 'the', 'as', 'is', 'and', ''} 
def rdd_helper(file):
    words_rdd = sc.textFile(file).flatMap(lambda line: line.split(" ")).filter(lambda word: word not in STOPSET)
    pair_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
    flipped = pair_rdd.map(lambda x: (x[1], x[0]))
    return flipped.mapValues(lambda x: (file, x))
    

def top(input_files):
    words_rdd = None
    for file in input_files:
        temp_rdd = rdd_helper(file)
        if words_rdd is not None:
            words_rdd = temp_rdd.union(words_rdd)
        else:
            words_rdd = temp_rdd
    return words_rdd.sortByKey(False)

In [23]:
InputPath = '/notebooks/InputFiles/'
input_files = [f for f in listdir(InputPath) if isfile(join(InputPath, f))]
input_files = [InputPath + x for x in input_files]
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)
output = top(input_files)

In [31]:
output.coalesce(1, shuffle = True).saveAsTextFile('notebooks/output')


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

